## 3.7 日付データの処理

### 共通処理

In [ ]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [ ]:
# ライブラリのimport

# NumPy用ライブラリ
import numpy as np

# Matplotlib中のpyplotライブラリのインポート
import matplotlib.pyplot as plt

# matplotlib日本語化対応ライブラリのインポート
import japanize_matplotlib

# pandas用ライブラリ
import pandas as pd

# データフレーム表示用関数
from IPython.display import display

# seaborn
import seaborn as sns

In [ ]:
# 表示オプション調整

# NumPy表示形式の設定
np.set_printoptions(
    suppress=True, precision=4, floatmode='fixed'
)

# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14

# サイズ設定
plt.rcParams['figure.figsize'] = (6, 6)

# 方眼表示ON
plt.rcParams['axes.grid'] = True

# データフレームでの表示精度
pd.options.display.float_format = '{:.4f}'.format

# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)

### 3.7.1 read_csv関数のparse_datesオプション

In [ ]:
# 支払情報
url1 = 'https://github.com/makaishi2/samples/raw/main/data/rental5-jp.csv'

# 支払情報
df1 = pd.read_csv(url1, parse_dates=[4])

# 貸出日時順にソート
df2 = df1.sort_values('貸出日時')
df2 = df2.reset_index(drop=True)

# データ型確認
print(df2.dtypes)

# 結果確認
display(df2.head(2))

### 3.7.2 日付集計用項目追加

In [ ]:
# 集計単位計算用関数
from datetime import datetime

# 週単位の日付作成
def conv_week_day(ts):

    # 日付を年、何週目か、週の何日目かに分解
    year, week, day = ts.isocalendar()

    # その週の1日目を表現する文字列
    str = f'{year} {week} 1'

    # 文字列をdatetime型に変換
    return datetime.strptime(str, "%Y %W %w")

# 日単位の日付作成
def conv_date(ts):

    # Timestampから文字列生成
    # str: YYYY-MM-DDThh:mm:ss
    str = ts.isoformat()

     # 先頭10文字をdatetime型に変換
    return datetime.strptime(str[:10], '%Y-%m-%d')

In [ ]:
df3 = df2.copy()

# 集計単位用の項目計算
df3.insert(4, '貸出日', df3['貸出日時'].map(conv_date))
df3.insert(5, '貸出週', df3['貸出日時'].map(conv_week_day))

# 結果確認
display(df3.head(2))

In [ ]:
# 参考

# 月単位の日付作成
def conv_month(ts):
    str = ts.isoformat()
    # str: YYYY-MM-DDThh:mm:ss
    return datetime.strptime(str[:7], '%Y-%m')

# 年単位の日付作成
def conv_year(ts):
    str = ts.isoformat()
    # str: YYYY-MM-DDThh:mm:ss
    return datetime.strptime(str[:4], "%Y")

### 3.7.3 週単位の集計

In [ ]:
# 週単位の集計
df4 = df3.groupby('貸出週')['レンタル代'].sum()

# 結果確認
display(df4)

In [ ]:
# 1週間単位のインデックス作成
date_index = pd.date_range(
    "2005-05-23", periods=14, freq="W-MON")
print(date_index)

In [ ]:
# 空の集計表作成
rent_fare = pd.Series(0, index=date_index)
print(rent_fare)

In [ ]:
# 売上集計
for ts in df4.index:
    rent_fare[ts] += df4[ts]

# 結果確認
print(rent_fare)

### 3.7.4 集計結果の可視化

In [ ]:
# サイズ設定
plt.rcParams['figure.figsize'] = (6, 6)

# 棒グラフ描画
rent_fare.plot(kind='bar')

# タイトル表示
plt.title('週単位の売上合計')
plt.show()

### 3.7.5 日付範囲指定

In [ ]:
# 日付範囲指定

# 開始日　datetime型で定義
sday = pd.to_datetime('2005-06-11')

# 終了日　datetime型で定義
eday = pd.to_datetime('2005-06-18')

# 顧客ID
uid = 459

# queryメソッドで絞り込み　検索条件はすべて変数による指定
x2 = df3.query(
    '顧客ID == @uid and 貸出日 >=@sday and 貸出日 <= @eday')

# 結果確認
display(x2)

### 3.7.6 日付への加減算(relativedelta関数)

In [ ]:
# ライブラリインポート
from dateutil.relativedelta import relativedelta

# 基準日付
t1 = x2['貸出日'].iloc[0]

# 4日後
ts = t1 + relativedelta(days=4)

# 14日後
te = t1 + relativedelta(days=14)

# 結果確認
print(t1, type(t1))
print(ts, type(ts))
print(te, type(te))

In [ ]:
# 計算結果を絞り込み条件に使う
x3 = df3.query('顧客ID == @uid and 貸出日 >=@ts and 貸出日 <= @te')

# 結果確認
display(x3)

### 演習問題
レンタルビデオ店では顧客離反防止施策の一環として、最近レンタルをしていない顧客をリストアップし、期間限定半額セールのDMを打つこととしました。  
顧客絞り込みの基準は「最終貸出日が2005-08-19以前」ということがすでに定まっているものとします。  
このとき、貸出情報から該当顧客を洗い出し、顧客情報を用いてDMの宛先をリスト変数形式で作成してください。  
リスト作成時は、実習で利用した貸出情報(df3)と下記の顧客情報(df5)を利用して下さい。

In [ ]:
# 顧客情報
url2 = 'https://github.com/makaishi2/samples/raw/main/data/customer-jp.csv'
# 顧客情報
df5 = pd.read_csv(url2)
display(df5.head(1))

In [ ]:
# df3から「顧客ID」と「貸出日」の列のみを抽出
df6 =

# 結果確認
display(df6.head(2))

In [ ]:
# 顧客IDごとの最終貸出日を求める
# (ヒント)顧客ごとグループ化して「貸出日」の最大値を求めればいい
df7 =

# 顧客IDをデータフレームの列に戻す
df8 =

# 結果確認
display(df8.head(2))

In [ ]:
# 最終貸出日が 2005-08-19以前の顧客を絞り込む

# 基準日の定義
ldate =

# queryメソッドで検索
df9 =

# 結果確認
display(df9)

In [ ]:
# メールアドレス一覧の作成

# df5から顧客ID、メールアドレスを抽出
df10 =

# df9と結合
df11 =

# 結果確認
display(df11.head(2))

In [ ]:
# メールアドレスをリスト形式で抽出
mlist =

# 結果確認
print(mlist)